In [1]:
library(RPostgreSQL)
library(Matching)
library(tidyverse)

Loading required package: DBI
Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks MASS::select()


In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
ventfreedays <- dbGetQuery(con, "select icustay_id, ventfreeday28 from subgroup;")
head(ventfreedays)

Warning message in postgresqlQuickSQL(conn, statement, ...):
“Could not create execute: select icustay_id, ventfreeday28 from subgroup;”

NULL

In [5]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))

In [6]:
names(full_data)

[1] "icustay_id"                     "hadm_id"                       
  [3] "subject_id"                     "first_careunit"                
  [5] "intime"                         "outtime"                       
  [7] "angus"                          "age"                           
  [9] "icu_order"                      "echo_time"                     
 [11] "echo_exclude"                   "echo_include"                  
 [13] "echo"                           "gender"                        
 [15] "weight"                         "saps"                          
 [17] "sofa"                           "elix_score"                    
 [19] "vent"                           "vaso"                          
 [21] "icu_adm_weekday"                "icu_adm_hour"                  
 [23] "deathtime"                      "mort_28_day"                   
 [25] "icu_los_day"                    "icd_chf"                       
 [27] "icd_afib"                       "icd_renal"                     
 [29] "icd_liver"                      "icd_copd"                      
 [31] "icd_cad"                        "icd_stroke"                    
 [33] "icd_malignancy"                 "vs_heart_rate_flag"            
 [35] "vs_heart_rate_first"            "vs_heart_rate_min"             
 [37] "vs_heart_rate_max"              "vs_cvp_flag"                   
 [39] "vs_cvp_first"                   "vs_cvp_min"                    
 [41] "vs_cvp_max"                     "vs_map_flag"                   
 [43] "vs_map_first"                   "vs_map_min"                    
 [45] "vs_map_max"                     "vs_temp_flag"                  
 [47] "vs_temp_first"                  "vs_temp_min"                   
 [49] "vs_temp_max"                    "lab_hemoglobin_flag"           
 [51] "lab_hemoglobin_first"           "lab_hemoglobin_min"            
 [53] "lab_hemoglobin_max"             "lab_hemoglobin_abnormal"       
 [55] "lab_platelet_flag"              "lab_platelet_first"            
 [57] "lab_platelet_min"               "lab_platelet_max"              
 [59] "lab_platelet_abnormal"          "lab_creatinine_kinase_flag"    
 [61] "lab_creatinine_kinase_first"    "lab_creatinine_kinase_min"     
 [63] "lab_creatinine_kinase_max"      "lab_creatinine_kinase_abnormal"
 [65] "lab_wbc_flag"                   "lab_wbc_first"                 
 [67] "lab_wbc_min"                    "lab_wbc_max"                   
 [69] "lab_wbc_abnormal"               "lab_ph_flag"                   
 [71] "lab_ph_first"                   "lab_ph_min"                    
 [73] "lab_ph_max"                     "lab_ph_abnormal"               
 [75] "lab_chloride_flag"              "lab_chloride_first"            
 [77] "lab_chloride_min"               "lab_chloride_max"              
 [79] "lab_chloride_abnormal"          "lab_sodium_flag"               
 [81] "lab_sodium_first"               "lab_sodium_min"                
 [83] "lab_sodium_max"                 "lab_sodium_abnormal"           
 [85] "lab_bun_flag"                   "lab_bun_first"                 
 [87] "lab_bun_min"                    "lab_bun_max"                   
 [89] "lab_bun_abnormal"               "lab_bicarbonate_flag"          
 [91] "lab_bicarbonate_first"          "lab_bicarbonate_min"           
 [93] "lab_bicarbonate_max"            "lab_bicarbonate_abnormal"      
 [95] "lab_bnp_flag"                   "lab_bnp_first"                 
 [97] "lab_bnp_min"                    "lab_bnp_max"                   
 [99] "lab_bnp_abnormal"               "lab_pco2_flag"                 
[101] "lab_pco2_first"                 "lab_pco2_min"                  
[103] "lab_pco2_max"                   "lab_pco2_abnormal"             
[105] "lab_creatinine_flag"            "lab_creatinine_first"          
[107] "lab_creatinine_min"             "lab_creatinine_max"            
[109] "lab_creatinine_abnormal"        "lab_potassium_flag"            
[111] "lab_potassium_first"            "la

In [7]:
set.seed(4958)

In [8]:
ps_matches <- Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1,
                    estimand = "ATT", caliper = 0.01,
                    exact = FALSE, replace = FALSE)

Warning message in Match(Y = NULL, Tr = full_data$echo_int, X = full_data$ps, M = 1, :
“replace==FALSE, but there are more (weighted) treated obs than control obs.  Some treated obs will not be matched.  You may want to estimate ATC instead.”

In [9]:
matches_df <-
full_data[ps_matches$index.treated, "icustay_id", drop = FALSE] %>%
mutate(match = full_data[ps_matches$index.control, "icustay_id"])

head(matches_df)

icustay_id,match
228416,276558
274174,297410
214954,213987
264413,299903
214106,292368
266275,270233


In [10]:
result <- full_data %>%
left_join(ventfreedays, by = "icustay_id") %>%
left_join(matches_df, by = "icustay_id") %>%
select(icustay_id, echo_int, icu_los_day, mort_28_day_int, ventfreeday28, match, ps) %>%
setNames(c("icustay_id", "Echo", "ICU length of stay", "28 day mortality",
           "Ventilation free days (28 days)", "Match id", "Propensity score"))

head(result)

ERROR: Error in UseMethod("tbl_vars"): no applicable method for 'tbl_vars' applied to an object of class "NULL"


In [ ]:
summary(ps_matches)

In [ ]:
tab <- table(full_data$mort_28_day[ps_matches$index.treated],
             full_data$mort_28_day[ps_matches$index.control],
             dnn = c("Echo", "Control"))
tab

In [ ]:
tab[2, 1] / tab[1, 2]
paste("95% Confint",
      round(exp(c(log(tab[2, 1] / tab[1, 2]) - qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]),
                  log(tab[2, 1] / tab[1, 2]) + qnorm(0.975) * sqrt(1 / tab[1, 2] + 1 / tab[2, 1]))), 7))

In [ ]:
mcnemar.test(tab)

In [ ]:
data.table::fwrite(result, file.path(data_dir, "table-5-columns.csv"))